- NoSQL 특강: 오전시간 

* SCRAPING

In [ ]:
# - Data Scraping - ancher을 만들어서 애들이 돌아다니면서 크롤링하는 것 
# - Data Crawling - 특정 일부분에 있는 애들을 가지고 오는 애들 

In [ ]:
# http://validator.w3.org/unicorn/?ucn_lang=ko  : 사이트 오류 알려주는 사이트 
# www.ppomppu.co.kr

In [ ]:
#revolution_main_table > tbody

In [1]:
import requests
import time

def download(method, url,
             param=None, data=None,
            timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url,
                     params=param, data=data,
                               headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<= e.response.status_code <600\
        and maxretries >0:
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

In [2]:
from bs4 import BeautifulSoup

In [218]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu"
html = download("get", url)
dom = BeautifulSoup(html.text, "lxml")

In [219]:
count = 0
for row in dom.select('td.list_vspace'):
    a = row.find('font',{"class":"list_title"})
    if a:
        cols = row.find_next_siblings("td",{"class":"eng list_vspace"})
        print(a.text ,cols[1].text, cols[2].text)

[홈플러스] 삼양 미역새콤비빔면 4입/튀김쫄면 4입(2,000/3,000)4만이상 무배  368
[티몬]부산출발 하노이 (1인 278,300/무료)  2193
[카카오 선물하기] 설레임130ml*24개(12900/무료) 1 - 0 1346
[G마켓] 쁘띠첼 미초 900mlx4개 (13,300원/무료배송)  478
[G마켓] 스파게티소스600gx2 +커리/쌀국수소스x1+면500g (9,900원 / 무배)  1274
[G9] 삼성공식인증점 갤럭시탭S5e 10.5 WiFi 128GB (488,700원/무료배송) + 상품평 작성시 해피머니1만원 0 - 3 3301
[G마켓] LG 휘센 제습기 DQ168PBE (368,470원/ 무료배송)  1462
[위메프] EVA 30CM 마사지 폼롤러 (5900/무료) 4 - 1 5455
[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)  2363
[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)  937
[Gsuite] Gsuite Basic India (3700원(월) / 무료) 3 - 0 6898
[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백 17 - 7 14065
[옥션] 식사에반하다25개입 (26,530원 / 무배) 1 - 0 6343
[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원) 36 - 0 11322
[티몬] 해피머니 10만원권 (90,900원/0원) 84 - 0 15611
[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송) 4 - 0 6889
[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650 7 - 0 5306
[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료) 100 - 17 42737


In [220]:
[_["tr"] for _ in dom.select("tr[class^=list]")]

KeyError: 'tr'

In [221]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]

[]

In [223]:
import re

In [224]:
for tr in dom.select("tr[class^=list]")[1:]: # tr에서 list로 설명되는 애 
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url, td[3].a.img["src"]))
    print(td[3].select ("td[valign=middle]> a")[0].text)
    temp= td[3].select("td[valign=middle]>a")[0].text
    print(re.findall(".+\((.+?)\).*", temp)[0].split("/")) # (가격) 가져오기 
    temp = td[5].text.split("-")
    print((0,0) if len(temp) < 2 else ",".join(temp))
    print(td[6].text.strip())
    print()

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/1/small_323891.jpg?t=20190716163637
[홈플러스] 삼양 미역새콤비빔면 4입/튀김쫄면 4입(2,000/3,000)4만이상 무배
['2,000', '3,000']
(0, 0)
368

http://static.ppomppu.co.kr/www/img/noimage/noimage_60x50.jpg
[티몬]부산출발 하노이 (1인 278,300/무료)
['1인 278,300', '무료']
(0, 0)
2193

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323889.jpg?t=20190716161444
[카카오 선물하기] 설레임130ml*24개(12900/무료)
['12900', '무료']
1 , 0
1346

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/8/small_323888.jpg?t=20190716161143
[G마켓] 쁘띠첼 미초 900mlx4개 (13,300원/무료배송)
['13,300원', '무료배송']
(0, 0)
478

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/7/small_323887.jpg?t=20190716154110
[G마켓] 스파게티소스600gx2 +커리/쌀국수소스x1+면500g (9,900원 / 무배)
['9,900원 ', ' 무배']
(0, 0)
1274

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/5/small_323885.jpg?t=20190716153432
[G9] 삼성공식인증점 갤럭시탭S5e 10.5 WiFi 128GB (488,700원/무료배송) + 상품평 작성시 해피머니1만원
['488,700원', '무료배송']
0 , 3
3301

http://cdn.ppomppu.co.kr/zboard/data/

In [ ]:
1. Focused Craling (10 페이지 내에서 )

In [102]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard"
html = download("get", url)
dom = BeautifulSoup(html.text, "lxml")

In [ ]:

#revolution_main_table > tbody > tr:nth-child(8) > td:nth-child(3) > a
for tr in dom.select("tr[class^=list]")[1:]: # tr에서 list로 설명되는 애 
    td = tr.find_all("td", recursive=False)

In [ ]:
for tr in dom.select("tr[class^=list]")[1:]: # tr에서 list로 설명되는 애 
    td = tr.find_all("td", recursive=False)

In [184]:
[_["href"] for _ in dom.select("#revolution_main_table td a") if checkBlog(_["href"]) if len(_["href"])>3]

['view.php?id=regulation&page=1&divpage=202&no=6',
 '/zboard/view.php?id=issue&no=160987',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543063',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543062',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543061',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543060',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543059',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543058',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543057',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543056',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543055',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543054',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543053',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543052',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543051',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543050',
 'zboard.php?id=freeboard&page=1&divpage=1221&no=6543049',
 'zboard.php?id=freeboar

In [188]:
[requests.compat.urljoin(url, _["href"]) for _ in dom.select("#revolution_main_table td a") if checkBlog(_["href"]) if len(_["href"])>3]

['http://www.ppomppu.co.kr/zboard/view.php?id=regulation&page=1&divpage=202&no=6',
 'http://www.ppomppu.co.kr/zboard/view.php?id=issue&no=160987',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543063',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543062',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543061',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543060',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543059',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543058',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543057',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543056',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543055',
 'http://www.ppomppu.co.kr/zboard/

In [196]:
[_["href"] for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(url, _["href"]))]

['http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=event',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=buy',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=help',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=etc_info',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=free_picture',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=news2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=review',
 'http://www.ppomppu.co.kr/recent_main_article.php?type=market',
 'http://www.ppomppu.co.kr/myinfo/env.php?cmd=env',
 'http://www.ppomppu.co.kr/myinfo/member_bookmark.php',
 'http://www.ppomppu.co.kr/index.php',
 '/zboard/zboard.php?id=freeboard',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu2',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu4',


In [167]:
for i in range (1,10):
    def checkBlog(url):
        return "www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page={0}".format(i)

In [209]:
def parseURL (seed):
    
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
   
    return [requests.compat.urljoin(url, _["href"]) 
              for _ in dom.select("#revolution_main_table td a") 
              if checkBlog(_["href"]) if len(_["href"])>3] # and not startwith.(#) # has_attr('class') 속성이 있는지 확인 

In [210]:
queue = list()
queue.extend([])
seen = list() # URL-seen 

while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    time.sleep(1) # 너무 빠르게 리퀘스트 보내지 않도록 '슬립'이 포함되어야 한다. 
    linkList = parseURL(baseURL)
    for link in linkList:  # 이미 있음에도 중복되는 애들을 제거해주어야 한다. 
        if link not in queue and not link in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

Queue: 31, Seen: 1
Queue: 30, Seen: 2
Queue: 306, Seen: 3
Queue: 313, Seen: 4


KeyboardInterrupt: 

In [211]:
queue

['http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543060',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543059',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543058',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543057',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543056',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543055',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543054',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543053',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543052',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543051',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543050',

In [234]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543053"
html = download("get", url)
dom = BeautifulSoup(html.text, "lxml")

In [ ]:
view_title2

In [215]:
for tr in dom.select("tr[class^=view]")[1:]: # tr에서 list로 설명되는 애 
    td = tr.find_all("td", recursive=False)
    print(td[4].select ("td[valign=middle]> a")[0].text)

In [3]:
def parseURL(seed):
    
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
   
    return [requests.compat.urljoin(seed, _.find_parent()["href"]) 
              for _ in dom.select("font.list_title") 
              if _.find_parent() and _.find_parent().has_attr("href")]

In [8]:
def parseContent(url):
    html = download("get", url)
    dom= BeautifulSoup(html.text, "lxml")
    
    return {"title": dom.select_one("font.view_title2").text.strip(), # strip: 공백 제거 
           "body":dom.select_one("td.board-contents").text.strip(),
           "comments": [_.text.strip() for _ in dom.select("div[id^=commentContent_]")]} # comprehension 

def parseContent(url):
    html = download("get", url)
    dom= BeautifulSoup(html.text, "lxml")
    
    return {"title": dom.select_one("font.view_title2").text.strip(), # strip: 공백 제거 
           "body": dom.select_one(".pic_bg .han"), # table- class  td- class
           "comments": [_.text.strip() for _ in dom.select("div[id^=commentContent_]")]} # comprehension 

In [28]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, param={"id":"freeboard"})
dom = BeautifulSoup(html.text, "lxml")

seed = list()
seed.extend([requests.compat.urljoin(url, _["href"])
            for _ in dom.select ("#page_list a")])
seen = list()
queue= list()

while seed:
    baseURL = seed.pop(0)
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 30, Seen: 1
Queue: 60, Seen: 2
Queue: 90, Seen: 3
Queue: 120, Seen: 4
Queue: 150, Seen: 5
Queue: 180, Seen: 6
Queue: 210, Seen: 7
Queue: 240, Seen: 8
Queue: 270, Seen: 9
Queue: 300, Seen: 10


In [29]:
contents= list()
while queue:
    baseURL=queue.pop(0)
    seen.append(baseURL)
    contents.append(parseContent(baseURL))

KeyboardInterrupt: 

In [30]:
len(contents)

12

In [31]:
import sqlite3

In [32]:
con = sqlite3.connect("bot.db")
cur = con.cursor() # cursor? 

# param 은 null일 수도 있따.
cur.executescript("""
    DROP TABLE IF EXISTS table3; 
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    );
""")
con.commit()

In [33]:
while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3 
        (title, content)
        VALUES(?,?)
    """, [content["title"], content["body"]])
    con.commit()
    
    cur.execute("""
        SELECT id
        FROM table3
        WHERE title=? AND content=?;
    """, [content["title"], content["body"]])
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4
            (table3_id, comment)
            VALUES(?,?)
        """, [table3_id, comment])
    con.commit()